# Online Production

In [1]:
#!pip install -U hopsworks --quiet

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
ms = project.get_model_serving()


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/716948
2024-05-13 18:00:09,935 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Fetch Model from Hopsworks Model Registry
deployment = ms.get_deployment(name="beerratingmodeldeployment1")
deployment.start()

Deployment is already running


## 🚀 Send an Inference Request

In [5]:
# Send inference requests to the deployed model as follows:
feature_view = fs.get_feature_view("user_beer_feature_view", 1)

In [6]:
# Initialize serving
feature_view.init_serving(1)    

2024-05-13 18:00:19,438 INFO: Default Online Store Client is set to sql.
2024-05-13 18:00:20,978 INFO: Initialising Vector Server Online SQL client
2024-05-13 18:00:26,330 INFO: Default Online Store Client is set to sql.
2024-05-13 18:00:27,653 INFO: Initialising Vector Server Online SQL client


In [12]:
import pandas as pd

#Load the simulatede data, stored as reviews.csv
reviews_df = pd.read_csv("reviews.csv")

# Inspect the first few rows of the DataFrame
print(reviews_df.head())

  beer_name                           beer_beerid  beer_brewerid  beer_abv  \
0      onto  b20ef6dd-851a-4747-b39b-d09f16d72400           1300       6.6   
1      left  44692c88-8fce-44c5-8d5f-2e44e36cad24           1590       7.8   
2      left  44692c88-8fce-44c5-8d5f-2e44e36cad24           1590       7.8   
3    theory  a21c1ff8-f53e-445e-8ca3-cb51474b4761           9620      10.7   
4    season  8c4e408f-a895-40e9-bfd7-95802f07144d           8358       5.4   

  beer_style  review_appearance  review_aroma  review_palate  review_taste  \
0      Stout                  8             6              9             6   
1      Lager                  1             9              5             6   
2      Lager                  6             2              9             7   
3        IPA                  9             7              8             8   
4      Lager                  9             2              1             6   

   review_overall          review_time review_profilename  \
0

In [15]:
# Group the data by beer ID and calculate aggregate statistics
aggregated_df = reviews_df.groupby("beer_beerid").agg({
    "beer_brewerid": "first",  # Brewer ID remains the same
    "beer_abv": "first",        # ABV remains the same
    "review_appearance": "mean",
    "review_aroma": "mean",
    "review_palate": "mean",
    "review_taste": "mean",
    "review_overall": "mean",
    # Count the number of reviews
    "beer_name": "size"  # Use any existing column to count the rows
}).reset_index()

# Rename the columns to match the model schema
renamed_columns = {
    "beer_brewerid": "beer_brewerid",
    "beer_abv": "beer_abv",
    "review_appearance": "avg_appearance",
    "review_aroma": "avg_aroma",
    "review_palate": "avg_palate",
    "review_taste": "avg_taste",
    "review_overall": "avg_overall",
    "beer_name": "review_count"  # Rename the column to "review_count"
}

aggregated_df.rename(columns=renamed_columns, inplace=True)

In [21]:
for index, row in aggregated_df.iterrows():
    # Extract relevant features from the row
    beer_brewerid = row['beer_brewerid']
    beer_abv = row['beer_abv']
    review_appearance = row['avg_appearance']  # Use the aggregated mean value
    review_aroma = row['avg_aroma']            # Use the aggregated mean value
    review_palate = row['avg_palate']          # Use the aggregated mean value
    review_taste = row['avg_taste']            # Use the aggregated mean value
    review_overall = row['avg_overall']        # Use the aggregated mean value
    avg_aroma = row['avg_aroma']              # Use the aggregated mean value
    avg_palate = row['avg_palate']            # Use the aggregated mean value
    avg_taste = row['avg_taste']              # Use the aggregated mean value
    avg_appearance = row['avg_appearance']    # Use the aggregated mean value
    review_count = row['review_count']         # Use the aggregated count value
    
    # Prepare input data in the format expected by the model serving engine
    input_data = [
        [
            beer_brewerid,
            beer_abv,
            review_appearance,
            review_aroma,
            review_palate,
            review_taste,
            review_overall,
            avg_aroma,
            avg_palate,
            avg_taste,
            avg_appearance,
            review_count
        ]
    ]
    
    # Send inference request to the deployed model
    prediction = deployment.predict(inputs=input_data)
    
    # Process or store the prediction as needed
    # For example, you could print the prediction
    print(f"Prediction for beer ID {row['beer_beerid']}: {prediction}")

Prediction for beer ID 318599ab-4f9b-4139-ae50-087695148ed7: {'predictions': [7.160097122192383]}
Prediction for beer ID 44692c88-8fce-44c5-8d5f-2e44e36cad24: {'predictions': [10.07424259185791]}
Prediction for beer ID 8c4e408f-a895-40e9-bfd7-95802f07144d: {'predictions': [7.372037410736084]}
Prediction for beer ID a21c1ff8-f53e-445e-8ca3-cb51474b4761: {'predictions': [5.650432109832764]}
Prediction for beer ID b20ef6dd-851a-4747-b39b-d09f16d72400: {'predictions': [6.734586715698242]}
Prediction for beer ID d8c01959-32bd-4e09-b0ed-2ada6f4141e4: {'predictions': [9.87358570098877]}
